```
1) wget으로 데이터 다운로드
$ wget http://files.grouplens.org/datasets/movielens/ml-1m.zip

2) 다운받은 데이터를 작업디렉토리로 이동
$ mv ml-1m.zip ~/aiffel/recommendata_iu/data

3) gzip으로 압축된 압축을 해제하면 tar 파일이 하나 나옵니다. 
$ unzip ml-1m.zip
```

In [168]:
import pandas as pd
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [169]:
using_cols = ['user_id', 'movie_id', 'rating']
ratings = ratings[using_cols]
ratings.tail()

,user_id,movie_id,rating
1000204,6040,1091,1
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4
1000208,6040,1097,4


In [170]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [171]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)

In [234]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


'Grumpier Old Men (1995)'

```
47::Seven (Se7en) (1995)::Crime|Thriller
628::Primal Fear (1996)::Drama|Thriller
2571::Matrix, The (1999)::Action|Sci-Fi|Thriller
1721::Titanic (1997)::Drama|Romance
2028::Saving Private Ryan (1998)::Action|Drama|War
```            

In [173]:
# 영화 수
ratings['movie_id'].nunique()

3628

In [174]:
# 사용자 수
ratings['user_id'].nunique()

6039

In [175]:
# 인기 많은 영화
movie_count = ratings.groupby('movie_id')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
480     2413
2762    2385
608     2371
110     2314
1580    2297
527     2257
1197    2252
2396    2213
1617    2210
318     2194
858     2167
1265    2121
1097    2102
2997    2066
2716    2051
296     2030
356     2022
1240    2019
1       2000
457     1941
Name: user_id, dtype: int64

### 모델 검증을 위한 사용자 초기 정보 세팅하기
 - 유튜브 추천 시스템 처럼, 5명의 영화 선택하면 좋아하는 영호들을 추천해주려고 함.

In [176]:

my_favorite = [47 , 628 ,2571 ,1721 ,2028]

my_playlist = pd.DataFrame({'user_id': [6041]*5, 'movie_id': my_favorite, 'count':[5]*5})

if not ratings.isin({'user_id':[6041]})['user_id'].any():  
    ratings = ratings.append(my_playlist)            


'''
47::Seven (Se7en) (1995)::Crime|Thriller
628::Primal Fear (1996)::Drama|Thriller
2571::Matrix, The (1999)::Action|Sci-Fi|Thriller
1721::Titanic (1997)::Drama|Romance
2028::Saving Private Ryan (1998)::Action|Drama|War
'''

ratings=ratings.reset_index(drop = True)
ratings.tail(10)       # 잘 추가되었는지 확인해 봅시다.


,user_id,movie_id,count
836473,6040,1090,3
836474,6040,1094,5
836475,6040,562,5
836476,6040,1096,4
836477,6040,1097,4
836478,6041,47,5
836479,6041,628,5
836480,6041,2571,5
836481,6041,1721,5
836482,6041,2028,5


### CSR (Compressed Sparse Row) Matrix

 - 데이터가 너무 클 때, 줄이기 위한 방법.

In [183]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = ratings['user_id'].unique()
movie_unique = ratings['movie_id'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [184]:
print(user_to_idx[6041])    # 358869명의 유저 중 마지막으로 추가된 유저이니 358868이 나와야 합니다. 
print(artist_to_idx[10])

6039
766


In [185]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_artist_data = ratings['movie_id'].map(artist_to_idx.get).dropna()
if len(temp_artist_data) == len(ratings):
    print('artist column indexing OK!!')
    ratings['movie_id'] = temp_artist_data
else:
    print('artist column indexing Fail!!')

ratings

user_id column indexing OK!!
artist column indexing OK!!


,user_id,movie_id,count
0,0,0,5
1,0,1,3
2,0,2,3
3,0,3,4
4,0,4,5
...,...,...,...
836478,6039,220,5
836479,6039,836,5
836480,6039,124,5
836481,6039,27,5


In [186]:
# 실습 위에 설명보고 이해해서 만들어보기
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_movie = ratings['movie_id'].nunique()



csr_data = csr_matrix((ratings['count'], (ratings.user_id, ratings.movie_id)), shape= (num_user, num_movie))
csr_data



<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

### MF 모델 학습하기

 - implicit 패키지로 학습심키기.
 - 이 패키지에 구현된 als(AlternatingLeastSquares)모델을 사용

In [187]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

1. factors : 유저와 아이템의 벡터를 몇 차원으로 할 것인지 
2. regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지 
3. use_gpu : GPU를 사용할 것인지 
4. iterations : epochs와 같은 의미입니다. 데이터를 몇 번 반복해서 학습할 것인지

1,4를 늘릴수록 학습데이터를 잘 학습하게 되지만 과적합의 우려가 있으니 좋은 값을 찾아야 합니다.

In [254]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=400, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

# 모델 훈련
als_model.fit(csr_data_transpose)

In [255]:
changhee, seven = user_to_idx[6041], artist_to_idx[47]
changhee_vector, seven_vector = als_model.user_factors[changhee], als_model.item_factors[seven]


In [256]:
# changhee과 seven를 내적하는 코드
np.dot(changhee_vector, seven_vector)

0.7326716

In [257]:
primal_fear = movie_to_idx[628]
primal_fear_vector = als_model.item_factors[primal_fear]
np.dot(changhee_vector, primal_fear_vector)

0.6115914

### 비슷한 영화 찾기 + 유저에게 추천하기

In [258]:
favorite_movie = 628
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie
for i in similar_movie:
    movie_name=movies.loc[i[0],'title']
    print(movie_name)

Big Squeeze, The (1996)
High Plains Drifter (1972)
Manchurian Candidate, The (1962)
Bridge at Remagen, The (1969)
Man Who Knew Too Much, The (1934)
Yojimbo (1961)
Good Earth, The (1937)
Butcher's Wife, The (1991)
Prince of Tides, The (1991)
King Creole (1958)
Alien: Resurrection (1997)
Porky's II: The Next Day (1983)
Shanghai Surprise (1986)
Erin Brockovich (2000)
Dersu Uzala (1974)
